<a href="https://www.kaggle.com/code/kcwong5/icr-identifying-age-related-conditions-initial?scriptVersionId=138399436" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import matplotlib.ticker as ticker
import plotly.graph_objects as go
from matplotlib.ticker import MaxNLocator

import time
from sklearn.preprocessing import LabelEncoder

import numpy as np
import optuna
from optuna.samplers import TPESampler

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split, StratifiedKFold

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Set notebook environment

In [2]:
# Change notebook environment

# https://ipython.readthedocs.io/en/stable/api/generated/IPython.core.interactiveshell.html
# https://stackoverflow.com/questions/36786722/how-to-display-full-output-in-jupyter-not-only-last-result
# Display full output in output cell, not only last result
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# https://www.kaggle.com/questions-and-answers/118932
# Max rows and columns of pandas dataframe 
pd.options.display.max_rows,pd.options.display.max_columns

# https://thispointer.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/
# Print all the contents of a pandas dataframe
pd.set_option('display.max_rows', None) # Print unlimited number of rows by setting to None, default is 10
pd.set_option('display.max_columns', None) # Do not truncate columns to display all of them by setting to None
pd.set_option('display.width', None) # Auto-detect the width of dataframe to display all columns in single line by setting to None
pd.set_option('display.max_colwidth', None) # Auto detect the max size of column and print contents of that column without truncation

# https://stackoverflow.com/questions/8924173/how-to-print-bold-text-in-python
start = "\033[1m" # Bold text
end = "\033[0;0m" # Reset 

import gc

In [3]:
# Free up memory that is no longer being used
gc.collect()

0

# Load datasets

In [4]:
train_df=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
greeks_df=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/greeks.csv")
test_df=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
sample_submission_df=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv")

# Data glance

In [5]:
print(start+"train.csv:"+end)
train_df.head()
print(start+"\ngreeks.csv:"+end)
greeks_df.head()
print(start+"\ntest.csv:"+end)
test_df.head()
print(start+"\nsample_submission_df.csv:"+end)
sample_submission_df.head()

train.csv:


,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1



greeks.csv:


,Id,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,B,C,G,D,3/19/2019
1,007255e47698,A,C,M,B,Unknown
2,013f2bd269f5,A,C,M,B,Unknown
3,043ac50845d5,A,C,M,B,Unknown
4,044fb8a146ec,D,B,F,B,3/25/2020



test.csv:


,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



sample_submission_df.csv:


,Id,class_0,class_1
0,00eed32682bb,0.5,0.5
1,010ebe33f668,0.5,0.5
2,02fa521e1838,0.5,0.5
3,040e15f562a2,0.5,0.5
4,046e85c7cc7f,0.5,0.5


# Check column names

In [6]:
print(start+"train.csv:"+end)
train_df.columns
print(start+"\ngreeks.csv:"+end)
greeks_df.columns
print(start+"\ntest.csv:"+end)
test_df.columns
print(start+"\nsample_submission_df.csv:"+end)
sample_submission_df.columns

train.csv:


Index(['Id', 'AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL', 'Class'],
      dtype='object')


greeks.csv:


Index(['Id', 'Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon'], dtype='object')


test.csv:


Index(['Id', 'AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
      dtype='object')


sample_submission_df.csv:


Index(['Id', 'class_0', 'class_1'], dtype='object')

# Assess data
- Attribute
- Summary statistics

In [7]:
print(start+"train:", train_df.shape,end)
print(start+"greeks:", greeks_df.shape,end)
print(start+"test:", test_df.shape,end)
print(start+"sample submission:", sample_submission_df.shape,end)

train: (617, 58) 
greeks: (617, 6) 
test: (5, 57) 
sample submission: (5, 3) 


In [8]:
print(start+"train.csv:"+end)
train_df.info()
print(start+"\ngreeks_df:"+end)
greeks_df.info()
print(start+"\ntest_df:"+end)
test_df.info()
print(start+"\nsample_submission_df:"+end)
sample_submission_df.info()

train.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 58 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      617 non-null    object 
 1   AB      617 non-null    float64
 2   AF      617 non-null    float64
 3   AH      617 non-null    float64
 4   AM      617 non-null    float64
 5   AR      617 non-null    float64
 6   AX      617 non-null    float64
 7   AY      617 non-null    float64
 8   AZ      617 non-null    float64
 9   BC      617 non-null    float64
 10  BD      617 non-null    float64
 11  BN      617 non-null    float64
 12  BP      617 non-null    float64
 13  BQ      557 non-null    float64
 14  BR      617 non-null    float64
 15  BZ      617 non-null    float64
 16  CB      615 non-null    float64
 17  CC      614 non-null    float64
 18  CD      617 non-null    float64
 19  CF      617 non-null    float64
 20  CH      617 non-null    float64
 21  CL      617 non-null    floa

In [9]:
print(start+"train_df:"+end)
train_df.describe()
print(start+"\ngreeks_df:"+end)
greeks_df.describe()
print(start+"\ntest_df:"+end)
test_df.describe()
print(start+"\nsample_submission_df:"+end)
sample_submission_df.describe()

train_df:


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
count,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,557.000000,617.000000,617.000000,615.000000,614.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,616.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,557.000000,617.000000,617.000000,616.000000,617.000000,617.000000,617.000000,616.000000,617.000000,615.000000,617.000000,617.000000,617.000000,617.000000,617.000000,616.000000,617.000000
mean,0.477149,3502.013221,118.624513,38.968552,10.128242,5.545576,0.060320,10.566447,8.053012,5350.388655,21.419492,231.322223,98.328737,1218.133238,550.632525,77.104151,0.688801,90.251735,11.241064,0.030615,1.403761,0.742262,36.917590,1.383792,27.165653,51.128326,401.901299,0.633884,0.367002,146.972099,94.795377,26.370568,1.802900,1.924830,26.388989,9.072700,3.064778,1731.248215,0.305107,69.582596,105.060712,69.117005,71.341526,6.930086,10306.810737,10.111079,5.433199,3.533905,0.421501,20.724856,131.714987,14679.595398,31.489716,50.584437,8.530961,0.175041
std,0.468388,2300.322717,127.838950,69.728226,10.518877,2.551696,0.416817,4.350645,65.166943,3021.326641,3.478278,183.992505,96.479371,7575.293707,2076.371275,159.049302,0.263994,51.585130,13.571133,0.014808,1.922210,0.281195,17.266347,0.538717,14.645993,21.210888,317.745623,1.912384,0.112989,86.084419,28.243187,8.038825,9.034721,1.484555,18.116679,6.200281,2.058344,1790.227476,1.847499,38.555707,68.445620,390.187057,165.551545,64.754262,11331.294051,2.934025,11.496257,50.181948,1.305365,9.991907,144.181524,19352.959387,9.864239,36.266251,10.327010,0.380310
min,0.081187,192.593280,85.200147,3.177522,8.138688,0.699861,0.025578,3.396778,1.229900,1693.624320,9.886800,72.948951,1.331155,51.216883,257.432377,12.499760,0.176874,23.387600,0.510888,0.003184,1.050225,0.069225,13.784111,0.137925,7.030640,6.906400,35.998895,0.238680,0.040995,60.232470,10.345600,6.339496,0.005518,1.743070,0.804068,4.926396,0.286201,185.594100,0.003042,5.394675,78.526968,3.828384,7.534128,0.296850,1563.136688,3.583450,0.173229,0.497060,0.067730,4.102182,72.611063,13.038894,9.432735,0.897628,0.001129,0.000000
25%,0.252107,2197.345480,85.200147,12.270314,8.138688,4.128294,0.025578,8.129580,1.229900,4155.702870,19.420500,156.847239,27.834425,424.990642,257.432377,23.317567,0.563688,64.724192,5.066306,0.023482,1.050225,0.589575,29.782467,1.070298,7.030640,37.942520,188.815690,0.238680,0.295164,102.703553,78.232240,20.888264,0.005518,1.743070,14.715792,5.965392,1.648679,1111.160625,0.003042,30.927468,78.526968,4.324656,25.815384,0.296850,5164.666260,8.523098,0.173229,0.497060,0.067730,14.036718,72.611063,2798.992584,25.034888,23.011684,0.124392,0.000000
50%,0.354659,3120.318960,85.200147,20.533110,8.138688,5.031912,0.025578,10.461320,1.229900,4997.960730,21.186000,193.908816,61.642115,627.417402,257.432377,42.554330,0.658715,79.819104,9.123000,0.027860,1.050225,0.730800,34.835130,1.351665,36.019104,49.180940,307.509595,0.238680,0.358023,130.050630,96.264960,25.248800,0.251741,1.743070,21.642456,8.149404,2.616119,1493.817413,0.085176,71.949306,78.526968,22.641144,36.394008,1.870155,7345.143424,9.945452,3.028141,1.131000,0.250601,18.771436,72.611063,7838.273610,30.608946,41.007968,0.337827,0.000000
75%,0.559763,4361.637390,113.739540,39.139886,8.138688,6.431634,0.036845,12.969516,5.081244,6035.885700,23.657700,247.803462,134.009015,975.649259,257.432377,77.310097,0.772206,99.813520,13.565901,0.034427,1.228445,0.859350,40.529401,1.660617,37.935832,61.408760,507.896200,0.238680,0.426348,165.836955,110.640680,30.544224,1.058690,1.743070,34.058344,10.503048,3.910070,1905.701475,0.237276,109.125159,112.766654,49.085352,56.714448,4.88021


greeks_df:


,Id,Alpha,Beta,Gamma,Delta,Epsilon
count,617,617,617,617,617,617
unique,617,4,3,8,4,198
top,000ff2bfdfe9,A,C,M,B,Unknown
freq,1,509,407,445,456,144



test_df:


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
count,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



sample_submission_df:


,class_0,class_1
count,5.0,5.0
mean,0.5,0.5
std,0.0,0.0
min,0.5,0.5
25%,0.5,0.5
50%,0.5,0.5
75%,0.5,0.5
max,0.5,0.5


# Check missing data

In [10]:
# Check for missing values

print(start+'Check missing values:'+end)

print(start+"\ngreeks_df:"+end)
print(greeks_df.isnull().sum())
print(start+"\ntrain_df:"+end)
print(train_df.isnull().sum())
print(start+"\ntest_df:"+end)
print(test_df.isnull().sum())
print(start+"\nsample_submission_df:"+end)
print(sample_submission_df.isnull().sum())

Check missing values:

greeks_df:
Id         0
Alpha      0
Beta       0
Gamma      0
Delta      0
Epsilon    0
dtype: int64

train_df:
Id        0
AB        0
AF        0
AH        0
AM        0
AR        0
AX        0
AY        0
AZ        0
BC        0
BD        0
BN        0
BP        0
BQ       60
BR        0
BZ        0
CB        2
CC        3
CD        0
CF        0
CH        0
CL        0
CR        0
CS        0
CU        0
CW        0
DA        0
DE        0
DF        0
DH        0
DI        0
DL        0
DN        0
DU        1
DV        0
DY        0
EB        0
EE        0
EG        0
EH        0
EJ        0
EL       60
EP        0
EU        0
FC        1
FD        0
FE        0
FI        0
FL        1
FR        0
FS        2
GB        0
GE        0
GF        0
GH        0
GI        0
GL        1
Class     0
dtype: int64

test_df:
Id     0
AB     0
AF     0
AH     0
AM     0
AR     0
AX     0
AY     0
AZ     0
BC     0
BD     0
BN     0
BP     0
BQ     0
BR     0
BZ     0
C

In [11]:
# Keep all rows with at least one missing data
train_df[train_df.isna().any(axis=1)]

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
8,0594b00fb30a,0.346113,3238.43674,85.200147,28.888816,8.138688,4.021986,0.025578,8.243016,3.626448,6569.370010,20.4798,135.881145,NaN,601.802912,257.432377,116.100712,0.855496,93.225352,14.566390,0.033830,1.050225,1.050375,29.914973,1.473039,43.015704,76.77356,231.134460,0.238680,0.330693,131.349555,98.16872,29.466032,0.613833,1.74307,7.200676,10.771632,1.342323,3004.926575,0.066924,B,NaN,78.526968,56.610456,35.300160,1.389258,3380.026318,11.450501,4.762291,1.18262,0.067730,17.245908,147.218610,4589.611956,29.771721,54.675576,0.073416,0
15,07760b4cf3f8,0.666588,3657.78160,304.283751,75.418144,8.138688,12.818973,0.025578,8.167392,1.229900,3597.873030,17.6550,227.744514,NaN,842.371642,257.432377,20.578649,0.651609,373.115464,6.925878,0.051342,1.050225,0.768900,13.784111,1.953018,7.030640,24.32760,647.221725,0.238680,0.333426,117.818145,75.31724,15.927216,0.268983,1.74307,6.446020,4.926396,3.115963,185.594100,0.006084,B,NaN,78.526968,41.357688,41.566224,2.719146,4143.635748,7.723713,67.872762,0.77720,0.582478,4.102182,72.611063,2218.449060,23.806958,35.843392,0.015231,0
18,0b2cc0b0e6c5,0.269199,1586.67784,85.200147,14.459893,8.138688,4.907886,0.032886,7.007824,3.521028,7125.170390,19.7736,176.541633,NaN,700.704062,257.432377,23.317567,0.574937,67.094664,35.649643,0.033432,1.050225,0.621375,33.660011,1.903365,36.658872,57.58308,176.650485,0.238680,0.475542,129.756960,85.36710,21.386416,0.372438,1.74307,17.932064,6.241044,3.970535,1389.371025,0.006084,B,NaN,102.487333,36.673488,32.329920,3.312846,4170.630264,13.727370,97.303580,2.73702,0.711165,16.362218,72.611063,3597.877440,31.330820,25.444796,0.011000,0
23,0cf6c827b8bb,0.371751,2354.54142,105.030540,5.150311,8.138688,10.409325,0.025578,11.526358,2.754976,5435.445190,19.7736,114.529302,NaN,220.658446,257.432377,134.694105,0.780432,56.891328,6.659790,0.023880,1.050225,0.571237,33.109065,1.125468,42.228736,30.65394,149.521590,0.238680,0.409950,96.227752,95.75828,21.270408,0.600039,1.74307,27.686442,4.926396,6.735801,8012.394450,0.121680,B,NaN,78.526968,21.775200,35.808528,2.493540,5709.501083,8.481751,4.648113,0.49706,0.866944,21.664358,198.469061,5987.708568,32.610844,15.019194,0.136552,0
24,0d1b855c7635,0.158101,3257.64549,138.368592,8.640630,18.385464,4.146012,0.182700,4.099451,7.836220,3633.584335,22.2453,170.308184,30.010903,355.723543,2568.243295,NaN,NaN,49.688740,11.840134,0.026069,1.050225,0.731025,57.192031,1.299254,36.817848,42.31334,356.052887,0.901017,0.379887,137.799000,77.89940,15.988632,0.005518,1.74307,23.394336,15.429444,1.130696,559.246525,0.003042,A,87.303431,304.176138,47.280036,117.815712,0.296850,13173.694370,10.358927,0.173229,1.82323,0.433472,8.311337,72.611063,1884.728169,48.039971,37.160256,21.978000,0
45,12d002d52230,1.491277,2434.32640,137.149332,124.242349,8.138688,4.730706,0.025578,4.487024,1.229900,2734.489610,20.4798,129.378519,NaN,2148.501967,257.432377,94.262896,0.715257,210.837232,10.924793,0.044974,1.196620,3.039675,22.717805,1.081332,39.496704,32.13804,556.277615,0.238680,0.308829,264.318060,102.47868,18.820592,0.151734,1.74307,39.264572,7.089204,5.865105,1000.589825,0.006084,B,NaN,102.199994,41.383008,17.867136,0.486834,6448.462858,4.917596,9.126892,2.44238,0.067730,30.919848,72.611063,819.350802,20.156657,31.537864,0.027000,0
46,1319c3883fff,0.243561,3316.81570,85.200147,13.406913,8.138688,3.809370,0.025578,10.505434,4.258968,3042.040690,18.3612,122.748723,NaN,366.787164,257.432377,38.234560,0.650385,85.471768,15.300792,0.018308,1.050225,0.895950,32.474431,1.130985,34.569920,51.10348,802.970055,0.238680,0.396285,129.113145,71.35708,29.186248,0.558657,1.74307,12.977388,4.926396,6.497972,1470.425350,0.042588,B,NaN,78.526968,3.828384,18.901008,3.449397,6946.192026,9.162606,17.726310,0.99586,0.291239,14.585536,160.164837,6209.544726,36.942088,33.093924,0.051333,0
48,135f1

# Check duplication

In [12]:
# Check for duplicates

print(start+'Train set: There are a total of', start+str(train_df.duplicated().sum())+end, start+'duplicate rows.\n'+end)
print(start+'Greeks: There are a total of', start+str(greeks_df.duplicated().sum())+end, start+'duplicate rows.\n'+end)
print(start+'Test set: There are a total of', start+str(test_df.duplicated().sum())+end, start+'duplicate rows.\n'+end)
print(start+'Submission: There are a total of', start+str(sample_submission_df.duplicated().sum())+end, start+'duplicate rows.\n'+end)

Train set: There are a total of 0 duplicate rows.

Greeks: There are a total of 0 duplicate rows.

Test set: There are a total of 0 duplicate rows.

Submission: There are a total of 0 duplicate rows.



In [13]:
# Check duplication by Id
train_df[train_df.Id.duplicated(keep=False)].sort_values("Id")
greeks_df[greeks_df.Id.duplicated(keep=False)].sort_values("Id")

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class


,Id,Alpha,Beta,Gamma,Delta,Epsilon


# Class imbalance

In [14]:
piefreq=train_df.Class.value_counts()

print('Class imbalance (counts): 1=Diagnosed, 0=Not Diagnosed')
print(piefreq,'\n')

# Imbalance class
print('Normalized class: Class 1, Class 0')
print(train_df.Class.value_counts(normalize=True))

# Pie chart of class imbalance
fig=go.Figure(data=[go.Pie(labels=['Class 0    (n=' + str(piefreq[0]) +')',
                                   'Class 1    (n=' + str(piefreq[1]) +')'],
                           values=train_df.Class.value_counts())])
fig.update_layout(title=dict(text="<b>Pie chart of 'Class'</b>",
                             y=0.85,x=0.4,
                             xanchor='center',
                             yanchor='top',
                             font=dict(size=14)
                            )
                  ,width=600
                  ,height=600)

# Delete piefreq dataframe to release memory
del piefreq, fig

Class imbalance (counts): 1=Diagnosed, 0=Not Diagnosed
0    509
1    108
Name: Class, dtype: int64 

Normalized class: Class 1, Class 0
0    0.824959
1    0.175041
Name: Class, dtype: float64


# Merge datasets by Id
- Train and Greek 

In [15]:
# Merge train and greeks dataframe by Id
merged_df = pd.merge(train_df, greeks_df, on="Id")

In [16]:
merged_df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1,B,C,G,D,3/19/2019
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0,A,C,M,B,Unknown
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0,A,C,M,B,Unknown
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0,A,C,M,B,Unknown
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1,D,B,F,B,3/25/2020


In [17]:
# Categorical feature
merged_df.EJ.value_counts()

B    395
A    222
Name: EJ, dtype: int64

In [18]:
# Categorical feature
print(start+"Alpha:"+end)
greeks_df[~greeks_df["Alpha"].str.isnumeric()]["Alpha"].value_counts()
print("\n"+start+"Beta:"+end)
greeks_df[~greeks_df["Beta"].str.isnumeric()]["Beta"].value_counts()
print("\n"+start+"Gamma:"+end)
greeks_df[~greeks_df["Gamma"].str.isnumeric()]["Gamma"].value_counts()
print("\n"+start+"Delta:"+end)
greeks_df[~greeks_df["Delta"].str.isnumeric()]["Delta"].value_counts()
print("\n"+start+"Epsilon:"+end)
greeks_df[~greeks_df["Epsilon"].str.isnumeric()]["Epsilon"].value_counts()

Alpha:


A    509
B     61
G     29
D     18
Name: Alpha, dtype: int64


Beta:


C    407
B    202
A      8
Name: Beta, dtype: int64


Gamma:


M    445
N     64
H     53
B     18
A     11
F     10
G      8
E      8
Name: Gamma, dtype: int64


Delta:


B    456
A     75
C     64
D     22
Name: Delta, dtype: int64


Epsilon:


Unknown       144
5/29/2019      24
6/26/2019      19
9/15/2020      17
8/29/2019      13
4/24/2019      11
9/8/2020       11
1/29/2019      10
1/31/2019      10
2/18/2019      10
4/30/2020       9
9/13/2020       8
7/30/2020       7
7/27/2020       6
6/11/2020       5
9/14/2020       5
11/27/2018      5
11/25/2019      5
9/11/2020       5
3/1/2019        5
12/10/2018      5
7/28/2020       5
5/6/2020        5
3/15/2019       5
2/28/2019       5
5/27/2020       4
6/21/2019       4
10/1/2019       4
4/29/2020       4
11/30/2018      4
12/4/2018       4
12/20/2018      4
3/13/2019       4
7/24/2020       4
4/28/2020       4
6/2/2020        4
9/7/2020        4
9/16/2020       3
7/16/2020       3
7/29/2020       3
12/24/2019      3
7/19/2019       3
3/7/2019        3
12/30/2019      3
4/9/2019        3
12/19/2019      3
12/9/2019       3
3/28/2019       2
5/2/2020        2
7/30/2019       2
7/15/2020       2
7/15/2019       2
1/28/2020       2
6/25/2020       2
2/22/2019       2
10/18/2019

In [19]:
# Print counts of 'Unknown' Epsilon
unknown_df=greeks_df.query("Epsilon=='Unknown'")
unknown_df.value_counts()

Id            Alpha  Beta  Gamma  Delta  Epsilon
007255e47698  A      C     M      B      Unknown    1
013f2bd269f5  A      C     M      B      Unknown    1
a7b1c7b83f53  A      C     M      B      Unknown    1
ab39fd701211  A      C     M      B      Unknown    1
ab4bc55cb79e  A      C     M      B      Unknown    1
adb67a0dd661  A      C     M      B      Unknown    1
aee064d35f72  A      C     M      B      Unknown    1
af0802c15f01  A      C     M      B      Unknown    1
afad421356e3  A      C     M      B      Unknown    1
b06b043e7abe  A      C     M      B      Unknown    1
b1056bf99b86  A      C     M      B      Unknown    1
b10bae274dfc  A      C     M      B      Unknown    1
b10f6f101138  A      C     M      B      Unknown    1
b301f38c110c  A      C     M      B      Unknown    1
b4d611fab6c5  A      C     M      B      Unknown    1
b874e25c20aa  A      C     M      B      Unknown    1
ba0c42e12f1f  A      C     M      B      Unknown    1
a633a258eeed  A      C     M     

# Feature Encoding

In [20]:
'''
le = LabelEncoder()

# Columns to encode
cat_features = ["EJ", "Alpha", "Beta", "Gamma", "Delta"] 
enc_df=merged_df.copy()

# Apply label encoder to each column
enc_df[cat_features] = enc_df[cat_features].apply(le.fit_transform) 

enc_df.info()
'''

'\nle = LabelEncoder()\n\n# Columns to encode\ncat_features = ["EJ", "Alpha", "Beta", "Gamma", "Delta"] \nenc_df=merged_df.copy()\n\n# Apply label encoder to each column\nenc_df[cat_features] = enc_df[cat_features].apply(le.fit_transform) \n\nenc_df.info()\n'

In [21]:
'''
print(start+"Before"+end)
train_df['EJ'].value_counts()
enc_df['EJ'].value_counts()

greeks_df['Alpha'].value_counts()
enc_df['Alpha'].value_counts()

greeks_df['Beta'].value_counts()
enc_df['Beta'].value_counts()

greeks_df['Gamma'].value_counts()
enc_df['Gamma'].value_counts()

greeks_df['Delta'].value_counts()
enc_df['Delta'].value_counts()
'''

'\nprint(start+"Before"+end)\ntrain_df[\'EJ\'].value_counts()\nenc_df[\'EJ\'].value_counts()\n\ngreeks_df[\'Alpha\'].value_counts()\nenc_df[\'Alpha\'].value_counts()\n\ngreeks_df[\'Beta\'].value_counts()\nenc_df[\'Beta\'].value_counts()\n\ngreeks_df[\'Gamma\'].value_counts()\nenc_df[\'Gamma\'].value_counts()\n\ngreeks_df[\'Delta\'].value_counts()\nenc_df[\'Delta\'].value_counts()\n'

# Modeling

## Evaluation metric

In [22]:
'''
def balanced_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # calculate the weights for each class to balance classes
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    # calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # calculate the summed log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    # calculate the weighted summed logarithmic loss
    # (factor of 2 included to give same result as LL with balanced input)
    balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    # return the average log loss
    return balanced_log_loss/(N_0+N_1)
'''

'\ndef balanced_log_loss(y_true, y_pred):\n    # y_true: correct labels 0, 1\n    # y_pred: predicted probabilities of class=1\n    # calculate the number of observations for each class\n    N_0 = np.sum(1 - y_true)\n    N_1 = np.sum(y_true)\n    # calculate the weights for each class to balance classes\n    w_0 = 1 / N_0\n    w_1 = 1 / N_1\n    # calculate the predicted probabilities for each class\n    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)\n    p_0 = 1 - p_1\n    # calculate the summed log loss for each class\n    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))\n    log_loss_1 = -np.sum(y_true * np.log(p_1))\n    # calculate the weighted summed logarithmic loss\n    # (factor of 2 included to give same result as LL with balanced input)\n    balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)\n    # return the average log loss\n    return balanced_log_loss/(N_0+N_1)\n'

In [23]:
'''
def balanced_log_loss_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    bll = balanced_log_loss(y_true, y_pred)
    return 'balanced_log_loss', bll, False
'''

"\ndef balanced_log_loss_eval(y_pred, dtrain):\n    y_true = dtrain.get_label()\n    bll = balanced_log_loss(y_true, y_pred)\n    return 'balanced_log_loss', bll, False\n"

In [24]:
# Drop features not found in test data, Separate the features and target variable
#X_df = enc_df.drop(["Id", "Alpha", "Beta", "Gamma", "Delta","Epsilon","Class"], axis=1)
#y_df = enc_df['Class']
X_df = merged_df.drop(["Id", "Alpha", "Beta", "Gamma", "Delta","Epsilon","Class"], axis=1)
y_df = merged_df['Class']

## 1)

In [25]:
'''
def objective(trial):
    train_x,valid_x,train_y,valid_y=train_test_split(X_df,y_df,test_size=0.2,random_state=42,shuffle=True,stratify=y_df)
    
    dtrain = lgb.Dataset(train_x,label=train_y,
                         feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                       'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                       'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                       'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                       'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                         categorical_feature=["EJ"])
    dvalid = lgb.Dataset(valid_x,label=valid_y,
                         feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                       'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                       'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                       'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                       'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                         categorical_feature=["EJ"])
    
    param = {"objective": "binary",
             "metric": "binary_logloss",
             "verbosity": -1,
             "boosting_type": "gbdt",
             "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
             "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
             "num_leaves": trial.suggest_int("num_leaves", 2, 256),
             "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
             "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
             "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
             "seed":42}
  
    # Train the LightGBM model with the custom evaluation function
    gbm = lgb.train(param, dtrain, valid_sets=[dvalid],feval=balanced_log_loss_eval,categorical_feature=["EJ"])
    preds = gbm.predict(valid_x)
    
    bll = balanced_log_loss(valid_y,preds)
    
    return bll


if __name__ == "__main__":
    # Reproducible optimization results
    # https://optuna.readthedocs.io/en/stable/faq.html#how-can-i-obtain-reproducible-optimization-results
    sampler = TPESampler(seed=42)  # Make the sampler behave in a deterministic way.
    study = optuna.create_study(direction="minimize",sampler=sampler)
    study.optimize(objective, n_trials=300)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
'''

'\ndef objective(trial):\n    train_x,valid_x,train_y,valid_y=train_test_split(X_df,y_df,test_size=0.2,random_state=42,shuffle=True,stratify=y_df)\n    \n    dtrain = lgb.Dataset(train_x,label=train_y,\n                         feature_name=[\'AB\', \'AF\', \'AH\', \'AM\', \'AR\', \'AX\', \'AY\', \'AZ\', \'BC\', \'BD \', \'BN\', \'BP\',\n                                       \'BQ\', \'BR\', \'BZ\', \'CB\', \'CC\', \'CD \', \'CF\', \'CH\', \'CL\', \'CR\', \'CS\', \'CU\',\n                                       \'CW \', \'DA\', \'DE\', \'DF\', \'DH\', \'DI\', \'DL\', \'DN\', \'DU\', \'DV\', \'DY\', \'EB\',\n                                       \'EE\', \'EG\', \'EH\', \'EJ\', \'EL\', \'EP\', \'EU\', \'FC\', \'FD \', \'FE\', \'FI\', \'FL\',\n                                       \'FR\', \'FS\', \'GB\', \'GE\', \'GF\', \'GH\', \'GI\', \'GL\'],\n                         categorical_feature=["EJ"])\n    dvalid = lgb.Dataset(valid_x,label=valid_y,\n                         feature_name=[\'

In [26]:
'''
Number of finished trials: 100
Best trial:
  Value: 0.20360767490328946
  Params: 
    lambda_l1: 0.00019581436316056585
    lambda_l2: 0.00032484094837833774
    num_leaves: 37
    feature_fraction: 0.8160074937253832
    bagging_fraction: 0.8503122588378281
    bagging_freq: 4
    min_child_samples: 56
    
Number of finished trials: 200
Best trial:
  Value: 0.19274028582043182
  Params: 
    lambda_l1: 0.00024753739779854604
    lambda_l2: 0.002320160687709684
    num_leaves: 70
    feature_fraction: 0.9151613574810961
    bagging_fraction: 0.7996847883349252
    bagging_freq: 7
    min_child_samples: 33
    
Number of finished trials: 300
Best trial:
  Value: 0.1758890868830618
  Params: 
    lambda_l1: 0.00026016459510927616
    lambda_l2: 0.0015693248796316954
    num_leaves: 65
    feature_fraction: 0.9633681484892685
    bagging_fraction: 0.7855075941459672
    bagging_freq: 7
    min_child_samples: 39
    
Number of finished trials: 400
Best trial:
  Value: 0.1758890868830618
  Params: 
    lambda_l1: 0.00026016459510927616
    lambda_l2: 0.0015693248796316954
    num_leaves: 65
    feature_fraction: 0.9633681484892685
    bagging_fraction: 0.7855075941459672
    bagging_freq: 7
    min_child_samples: 39
'''

'\nNumber of finished trials: 100\nBest trial:\n  Value: 0.20360767490328946\n  Params: \n    lambda_l1: 0.00019581436316056585\n    lambda_l2: 0.00032484094837833774\n    num_leaves: 37\n    feature_fraction: 0.8160074937253832\n    bagging_fraction: 0.8503122588378281\n    bagging_freq: 4\n    min_child_samples: 56\n    \nNumber of finished trials: 200\nBest trial:\n  Value: 0.19274028582043182\n  Params: \n    lambda_l1: 0.00024753739779854604\n    lambda_l2: 0.002320160687709684\n    num_leaves: 70\n    feature_fraction: 0.9151613574810961\n    bagging_fraction: 0.7996847883349252\n    bagging_freq: 7\n    min_child_samples: 33\n    \nNumber of finished trials: 300\nBest trial:\n  Value: 0.1758890868830618\n  Params: \n    lambda_l1: 0.00026016459510927616\n    lambda_l2: 0.0015693248796316954\n    num_leaves: 65\n    feature_fraction: 0.9633681484892685\n    bagging_fraction: 0.7855075941459672\n    bagging_freq: 7\n    min_child_samples: 39\n    \nNumber of finished trials: 400\n

### Submission

In [27]:
'''
# Get the best hyperparameters from the Optuna study
best_params = study.best_params

# Create a LightGBM dataset from the entire training data
dtrain = lgb.Dataset(X_df, label=y_df,params={'force_col_wise': True})

# Merge the best_params dictionary with the force_col_wise parameter
params = {**best_params, **{'force_col_wise': True}}

# Train a LightGBM model using the merged parameters
gbm = lgb.train(params, dtrain)

submission_df=test_df.drop(["Id"], axis=1)
submission_df['EJ'] = le.transform(submission_df['EJ']) # Replaces the original categorical values with their corresponding integer encodings.

# Make predictions on unseen data
preds = gbm.predict(submission_df)

# The predictions are probabilities for class 1
prob_class_1 = preds

# To get probabilities for class 0, subtract from 1
prob_class_0 = 1 - prob_class_1

# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/412946
# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/409801
sample = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
sample['class_1'] = np.clip(prob_class_1,1e-15,1 - 1e-15)
sample['class_0'] = prob_class_0
sample.to_csv('submission.csv', index = False)

# Display the contents of the submission.csv
display(pd.read_csv('submission.csv'))
'''

'\n# Get the best hyperparameters from the Optuna study\nbest_params = study.best_params\n\n# Create a LightGBM dataset from the entire training data\ndtrain = lgb.Dataset(X_df, label=y_df,params={\'force_col_wise\': True})\n\n# Merge the best_params dictionary with the force_col_wise parameter\nparams = {**best_params, **{\'force_col_wise\': True}}\n\n# Train a LightGBM model using the merged parameters\ngbm = lgb.train(params, dtrain)\n\nsubmission_df=test_df.drop(["Id"], axis=1)\nsubmission_df[\'EJ\'] = le.transform(submission_df[\'EJ\']) # Replaces the original categorical values with their corresponding integer encodings.\n\n# Make predictions on unseen data\npreds = gbm.predict(submission_df)\n\n# The predictions are probabilities for class 1\nprob_class_1 = preds\n\n# To get probabilities for class 0, subtract from 1\nprob_class_0 = 1 - prob_class_1\n\n# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/412946\n# https://www.kaggle.com/competitio

In [28]:
'''
Id	class_0	class_1
0	00eed32682bb	0.544107	0.455893
1	010ebe33f668	0.544107	0.455893
2	02fa521e1838	0.544107	0.455893
3	040e15f562a2	0.544107	0.455893
4	046e85c7cc7f	0.544107	0.455893
'''

'\nId\tclass_0\tclass_1\n0\t00eed32682bb\t0.544107\t0.455893\n1\t010ebe33f668\t0.544107\t0.455893\n2\t02fa521e1838\t0.544107\t0.455893\n3\t040e15f562a2\t0.544107\t0.455893\n4\t046e85c7cc7f\t0.544107\t0.455893\n'

In [29]:
'''
del dtrain, gbm, preds, study, trial
del submission_df, best_params
del prob_class_1, prob_class_0, sample
'''

'\ndel dtrain, gbm, preds, study, trial\ndel submission_df, best_params\ndel prob_class_1, prob_class_0, sample\n'

## 2) 

In [30]:
'''
X = X_df.to_numpy()
y = y_df.to_numpy()

def objective(trial):
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    param = {"objective": "binary",
             "metric": "binary_logloss",
             "verbosity": -1,
             "boosting_type": "gbdt",
             "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
             "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
             "num_leaves": trial.suggest_int("num_leaves", 2, 256),
             "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
             "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
             "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
             "seed": 42,
             "learning_rate":trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True),
            }

    for train_index, valid_index in skf.split(X, y):
        train_x, valid_x = X[train_index], X[valid_index]
        train_y, valid_y = y[train_index], y[valid_index]

        dtrain = lgb.Dataset(train_x, label=train_y,
                     feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                   'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                   'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                   'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                   'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                     categorical_feature=["EJ"])
                                   
        dvalid = lgb.Dataset(valid_x, label=valid_y,
                     feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                   'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                   'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                   'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                   'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                     categorical_feature=["EJ"])
                                   
        # Train the LightGBM model with the custom evaluation function
        gbm = lgb.train(param, dtrain, valid_sets=[dvalid], feval=balanced_log_loss_eval)
        preds = gbm.predict(valid_x)

        bll = balanced_log_loss(valid_y, preds)
        scores.append(bll)

    return np.mean(scores)

if __name__ == "__main__":
    
    sampler = TPESampler(seed=42)  # Make the sampler behave in a deterministic way.
    study = optuna.create_study(direction="minimize",sampler=sampler)
    study.optimize(objective, n_trials=300)


    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
'''

'\nX = X_df.to_numpy()\ny = y_df.to_numpy()\n\ndef objective(trial):\n    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)\n    scores = []\n    \n    param = {"objective": "binary",\n             "metric": "binary_logloss",\n             "verbosity": -1,\n             "boosting_type": "gbdt",\n             "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),\n             "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),\n             "num_leaves": trial.suggest_int("num_leaves", 2, 256),\n             "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),\n             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),\n             "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),\n             "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),\n             "seed": 42,\n             "learning_rate":trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True),\n            }

In [31]:
'''
Number of finished trials: 100
Best trial:
  Value: 0.22606619458720836
  Params: 
    lambda_l1: 0.0009562865883307864
    lambda_l2: 1.0501184667562712e-05
    num_leaves: 196
    feature_fraction: 0.8877290003776485
    bagging_fraction: 0.855495279749778
    bagging_freq: 2
    min_child_samples: 73
    learning_rate: 0.07893136657351986
    
Number of finished trials: 200
Best trial:
  Value: 0.2165360692415715
  Params: 
    lambda_l1: 0.0005763971757861298
    lambda_l2: 5.567550574259273e-05
    num_leaves: 197
    feature_fraction: 0.8936258395576222
    bagging_fraction: 0.8839392483012035
    bagging_freq: 2
    min_child_samples: 71
    learning_rate: 0.09824604022136249
    
Number of finished trials: 300
Best trial:
  Value: 0.20683746255683158
  Params: 
    lambda_l1: 7.940111512618743e-06
    lambda_l2: 1.774347600050857e-06
    num_leaves: 215
    feature_fraction: 0.9988662903874678
    bagging_fraction: 0.8490318978526208
    bagging_freq: 1
    min_child_samples: 64
    learning_rate: 0.07562682408122855
    
Number of finished trials: 400
Best trial:
  Value: 0.20683746255683158
  Params: 
    lambda_l1: 7.940111512618743e-06
    lambda_l2: 1.774347600050857e-06
    num_leaves: 215
    feature_fraction: 0.9988662903874678
    bagging_fraction: 0.8490318978526208
    bagging_freq: 1
    min_child_samples: 64
    learning_rate: 0.07562682408122855
'''

'\nNumber of finished trials: 100\nBest trial:\n  Value: 0.22606619458720836\n  Params: \n    lambda_l1: 0.0009562865883307864\n    lambda_l2: 1.0501184667562712e-05\n    num_leaves: 196\n    feature_fraction: 0.8877290003776485\n    bagging_fraction: 0.855495279749778\n    bagging_freq: 2\n    min_child_samples: 73\n    learning_rate: 0.07893136657351986\n    \nNumber of finished trials: 200\nBest trial:\n  Value: 0.2165360692415715\n  Params: \n    lambda_l1: 0.0005763971757861298\n    lambda_l2: 5.567550574259273e-05\n    num_leaves: 197\n    feature_fraction: 0.8936258395576222\n    bagging_fraction: 0.8839392483012035\n    bagging_freq: 2\n    min_child_samples: 71\n    learning_rate: 0.09824604022136249\n    \nNumber of finished trials: 300\nBest trial:\n  Value: 0.20683746255683158\n  Params: \n    lambda_l1: 7.940111512618743e-06\n    lambda_l2: 1.774347600050857e-06\n    num_leaves: 215\n    feature_fraction: 0.9988662903874678\n    bagging_fraction: 0.8490318978526208\n    ba

### Submission

In [32]:
'''
# Get the best hyperparameters from the Optuna study
best_params = study.best_params

# Create a LightGBM dataset from the entire training data
dtrain = lgb.Dataset(X_df, label=y_df,params={'force_col_wise': True})

# Merge the best_params dictionary with the force_col_wise parameter
params = {**best_params, **{'force_col_wise': True}}

# Train a LightGBM model using the merged parameters
gbm = lgb.train(params, dtrain)

submission_df=test_df.drop(["Id"], axis=1)
submission_df['EJ'] = le.transform(submission_df['EJ']) # Replaces the original categorical values with their corresponding integer encodings.

# Make predictions on unseen data
preds = gbm.predict(submission_df)

# The predictions are probabilities for class 1
prob_class_1 = preds

# To get probabilities for class 0, subtract from 1
prob_class_0 = 1 - prob_class_1

# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/412946
# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/409801
sample = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
sample['class_1'] = np.clip(prob_class_1,1e-15,1 - 1e-15)
sample['class_0'] = prob_class_0
sample.to_csv('submission.csv', index = False)

# Display the contents of the submission.csv
display(pd.read_csv('submission.csv'))
'''

'\n# Get the best hyperparameters from the Optuna study\nbest_params = study.best_params\n\n# Create a LightGBM dataset from the entire training data\ndtrain = lgb.Dataset(X_df, label=y_df,params={\'force_col_wise\': True})\n\n# Merge the best_params dictionary with the force_col_wise parameter\nparams = {**best_params, **{\'force_col_wise\': True}}\n\n# Train a LightGBM model using the merged parameters\ngbm = lgb.train(params, dtrain)\n\nsubmission_df=test_df.drop(["Id"], axis=1)\nsubmission_df[\'EJ\'] = le.transform(submission_df[\'EJ\']) # Replaces the original categorical values with their corresponding integer encodings.\n\n# Make predictions on unseen data\npreds = gbm.predict(submission_df)\n\n# The predictions are probabilities for class 1\nprob_class_1 = preds\n\n# To get probabilities for class 0, subtract from 1\nprob_class_0 = 1 - prob_class_1\n\n# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/412946\n# https://www.kaggle.com/competitio

In [33]:
'''
Number of finished trials: 100:
Id	class_0	class_1
0	00eed32682bb	0.595334	0.404666
1	010ebe33f668	0.595334	0.404666
2	02fa521e1838	0.595334	0.404666
3	040e15f562a2	0.595334	0.404666
4	046e85c7cc7f	0.595334	0.404666

Number of finished trials: 300:
	Id	class_0	class_1
0	00eed32682bb	0.581345	0.418655
1	010ebe33f668	0.581345	0.418655
2	02fa521e1838	0.581345	0.418655
3	040e15f562a2	0.581345	0.418655
4	046e85c7cc7f	0.581345	0.418655
'''

'\nNumber of finished trials: 100:\nId\tclass_0\tclass_1\n0\t00eed32682bb\t0.595334\t0.404666\n1\t010ebe33f668\t0.595334\t0.404666\n2\t02fa521e1838\t0.595334\t0.404666\n3\t040e15f562a2\t0.595334\t0.404666\n4\t046e85c7cc7f\t0.595334\t0.404666\n\nNumber of finished trials: 300:\n\tId\tclass_0\tclass_1\n0\t00eed32682bb\t0.581345\t0.418655\n1\t010ebe33f668\t0.581345\t0.418655\n2\t02fa521e1838\t0.581345\t0.418655\n3\t040e15f562a2\t0.581345\t0.418655\n4\t046e85c7cc7f\t0.581345\t0.418655\n'

In [34]:
'''
del dtrain, gbm, preds, study, trial
del submission_df, best_params
del prob_class_1, prob_class_0, sample
'''

'\ndel dtrain, gbm, preds, study, trial\ndel submission_df, best_params\ndel prob_class_1, prob_class_0, sample\n'

## 3) LightGBMTunerCV

In [35]:
'''
import optuna.integration.lightgbm as lgb
from lightgbm import early_stopping
from lightgbm import log_evaluation
import sklearn.datasets
from sklearn.model_selection import StratifiedKFold

def objective(trial):
    dtrain = lgb.Dataset(X_df,label=y_df,                      
                         feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                       'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                       'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                       'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                       'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                         categorical_feature=["EJ"])

    param = {"objective": "binary",
             "metric": "binary_logloss",
             "verbosity": -1,
             "boosting_type": "gbdt",
             "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
             "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
             "num_leaves": trial.suggest_int("num_leaves", 2, 256),
             "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
             "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
             "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
             "seed": 42}

    tuner = lgb.LightGBMTunerCV(param,
                                dtrain,
                                folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                                callbacks=[early_stopping(100),
                                           log_evaluation(100)],
                                optuna_seed=42,
                                feval=balanced_log_loss_eval,
                                categorical_feature=["EJ"]
                                )
    tuner.run()

    return tuner.best_score

sampler = TPESampler(seed=42)  # Make the sampler behave in a deterministic way.
study = optuna.create_study(direction="minimize",sampler=sampler)
study.optimize(objective, n_trials=100)

print("Best score:", study.best_value)
best_params = study.best_params
print("Best params:", best_params)
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))
'''

'\nimport optuna.integration.lightgbm as lgb\nfrom lightgbm import early_stopping\nfrom lightgbm import log_evaluation\nimport sklearn.datasets\nfrom sklearn.model_selection import StratifiedKFold\n\ndef objective(trial):\n    dtrain = lgb.Dataset(X_df,label=y_df,                      \n                         feature_name=[\'AB\', \'AF\', \'AH\', \'AM\', \'AR\', \'AX\', \'AY\', \'AZ\', \'BC\', \'BD \', \'BN\', \'BP\',\n                                       \'BQ\', \'BR\', \'BZ\', \'CB\', \'CC\', \'CD \', \'CF\', \'CH\', \'CL\', \'CR\', \'CS\', \'CU\',\n                                       \'CW \', \'DA\', \'DE\', \'DF\', \'DH\', \'DI\', \'DL\', \'DN\', \'DU\', \'DV\', \'DY\', \'EB\',\n                                       \'EE\', \'EG\', \'EH\', \'EJ\', \'EL\', \'EP\', \'EU\', \'FC\', \'FD \', \'FE\', \'FI\', \'FL\',\n                                       \'FR\', \'FS\', \'GB\', \'GE\', \'GF\', \'GH\', \'GI\', \'GL\'],\n                         categorical_feature=["EJ"])\n\n   

In [36]:
# ** Long running time

'''
Best score: 0.16610925806720525
Best params: {'lambda_l1': 2.348881295853308e-05, 'lambda_l2': 3.6010467344475403, 'num_leaves': 188, 'feature_fraction': 0.759195090518222, 'bagging_fraction': 0.4936111842654619, 'bagging_freq': 2, 'min_child_samples': 10}
  Params: 
    lambda_l1: 2.348881295853308e-05
    lambda_l2: 3.6010467344475403
    num_leaves: 188
    feature_fraction: 0.759195090518222
    bagging_fraction: 0.4936111842654619
    bagging_freq: 2
    min_child_samples: 10
'''

'''
Best score: 0.16501718423663816
Best params: {'lambda_l1': 0.018919430862786926, 'lambda_l2': 2.0866860111902543, 'num_leaves': 146, 'feature_fraction': 0.7052977973517935, 'bagging_fraction': 0.7029310340616282, 'bagging_freq': 2, 'min_child_samples': 6}
  Params: 
    lambda_l1: 0.018919430862786926
    lambda_l2: 2.0866860111902543
    num_leaves: 146
    feature_fraction: 0.7052977973517935
    bagging_fraction: 0.7029310340616282
    bagging_freq: 2
    min_child_samples: 6
'''

"\nBest score: 0.16610925806720525\nBest params: {'lambda_l1': 2.348881295853308e-05, 'lambda_l2': 3.6010467344475403, 'num_leaves': 188, 'feature_fraction': 0.759195090518222, 'bagging_fraction': 0.4936111842654619, 'bagging_freq': 2, 'min_child_samples': 10}\n  Params: \n    lambda_l1: 2.348881295853308e-05\n    lambda_l2: 3.6010467344475403\n    num_leaves: 188\n    feature_fraction: 0.759195090518222\n    bagging_fraction: 0.4936111842654619\n    bagging_freq: 2\n    min_child_samples: 10\n"

"\nBest score: 0.16501718423663816\nBest params: {'lambda_l1': 0.018919430862786926, 'lambda_l2': 2.0866860111902543, 'num_leaves': 146, 'feature_fraction': 0.7052977973517935, 'bagging_fraction': 0.7029310340616282, 'bagging_freq': 2, 'min_child_samples': 6}\n  Params: \n    lambda_l1: 0.018919430862786926\n    lambda_l2: 2.0866860111902543\n    num_leaves: 146\n    feature_fraction: 0.7052977973517935\n    bagging_fraction: 0.7029310340616282\n    bagging_freq: 2\n    min_child_samples: 6\n"

### Submission

In [37]:
'''
submission_df=test_df.drop(["Id"], axis=1)
submission_df['EJ'] = le.transform(submission_df['EJ']) # Replaces the original categorical values with their corresponding integer encodings.

# Create a LightGBM dataset from the entire training data
dtrain = lgb.Dataset(X_df,label=y_df,params={'force_col_wise': True})

gbm = lgb.train(best_params,dtrain)
preds = gbm.predict_proba(submission_df)
'''

'\nsubmission_df=test_df.drop(["Id"], axis=1)\nsubmission_df[\'EJ\'] = le.transform(submission_df[\'EJ\']) # Replaces the original categorical values with their corresponding integer encodings.\n\n# Create a LightGBM dataset from the entire training data\ndtrain = lgb.Dataset(X_df,label=y_df,params={\'force_col_wise\': True})\n\ngbm = lgb.train(best_params,dtrain)\npreds = gbm.predict_proba(submission_df)\n'

In [38]:
'''
import optuna.integration.lightgbm as lgb

from lightgbm import early_stopping
from lightgbm import log_evaluation
import sklearn.datasets
#from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    dtrain = lgb.Dataset(X_df, label=y_df,
                         feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                       'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                       'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                       'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                       'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                         categorical_feature=["EJ"])

    param = {"objective": "binary",
             "metric": "binary_logloss",
             "verbosity": -1,
             "boosting_type": "gbdt",
             "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
             "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
             "num_leaves": trial.suggest_int("num_leaves", 2, 256),
             "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
             "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
             "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
             "seed":42,
             "use_missing":True}
        
    tuner = lgb.LightGBMTunerCV(param,
                                dtrain,
                                folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                                callbacks=[early_stopping(100), 
                                           log_evaluation(100)],
                                optuna_seed=42,
                                feval=balanced_log_loss_eval,
                                feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                              'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                              'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                              'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                              'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                                categorical_feature=["EJ"]
                               )
   
    tuner.run()

    print("Best score:", tuner.best_score)
    best_params = tuner.best_params
    print("Best params:", best_params)
    print("  Params: ")
    for key, value in best_params.items():
        print("    {}: {}".format(key, value))
'''

'\nimport optuna.integration.lightgbm as lgb\n\nfrom lightgbm import early_stopping\nfrom lightgbm import log_evaluation\nimport sklearn.datasets\n#from sklearn.model_selection import KFold\nfrom sklearn.model_selection import StratifiedKFold\n\nif __name__ == "__main__":\n    dtrain = lgb.Dataset(X_df, label=y_df,\n                         feature_name=[\'AB\', \'AF\', \'AH\', \'AM\', \'AR\', \'AX\', \'AY\', \'AZ\', \'BC\', \'BD \', \'BN\', \'BP\',\n                                       \'BQ\', \'BR\', \'BZ\', \'CB\', \'CC\', \'CD \', \'CF\', \'CH\', \'CL\', \'CR\', \'CS\', \'CU\',\n                                       \'CW \', \'DA\', \'DE\', \'DF\', \'DH\', \'DI\', \'DL\', \'DN\', \'DU\', \'DV\', \'DY\', \'EB\',\n                                       \'EE\', \'EG\', \'EH\', \'EJ\', \'EL\', \'EP\', \'EU\', \'FC\', \'FD \', \'FE\', \'FI\', \'FL\',\n                                       \'FR\', \'FS\', \'GB\', \'GE\', \'GF\', \'GH\', \'GI\', \'GL\'],\n                         cate

In [39]:
'''
import optuna.integration.lightgbm as lgb
from lightgbm import early_stopping
from lightgbm import log_evaluation
import sklearn.datasets
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    # Dataset initialization
    dtrain = lgb.Dataset(X_df, label=y_df,
                         feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                       'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                       'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                       'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                       'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                         categorical_feature=["EJ"])

    def objective(trial):
        param = {
            "objective": "binary",
            "metric": "binary_logloss",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 2, 256),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
            "seed": 42,
            "use_missing": True
        }

        tuner = lgb.LightGBMTunerCV(param,
                                    dtrain,
                                    folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                                    callbacks=[early_stopping(100),
                                               log_evaluation(100)],
                                    optuna_seed=42,
                                    feval=balanced_log_loss_eval,
                                    feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD', 'BN', 'BP',
                                                  'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                                  'CW', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                                  'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD', 'FE', 'FI', 'FL',
                                                  'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                                    categorical_feature=["EJ"]
                                    )
        tuner.run()

        return tuner.best_score

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100)

    print("Best score:", study.best_value)
    best_params = study.best_params
    print("Best params:", best_params)
    print("  Params: ")
    for key, value in best_params.items():
        print("    {}: {}".format(key, value))
'''

'\nimport optuna.integration.lightgbm as lgb\nfrom lightgbm import early_stopping\nfrom lightgbm import log_evaluation\nimport sklearn.datasets\nfrom sklearn.model_selection import StratifiedKFold\n\nif __name__ == "__main__":\n    # Dataset initialization\n    dtrain = lgb.Dataset(X_df, label=y_df,\n                         feature_name=[\'AB\', \'AF\', \'AH\', \'AM\', \'AR\', \'AX\', \'AY\', \'AZ\', \'BC\', \'BD \', \'BN\', \'BP\',\n                                       \'BQ\', \'BR\', \'BZ\', \'CB\', \'CC\', \'CD \', \'CF\', \'CH\', \'CL\', \'CR\', \'CS\', \'CU\',\n                                       \'CW \', \'DA\', \'DE\', \'DF\', \'DH\', \'DI\', \'DL\', \'DN\', \'DU\', \'DV\', \'DY\', \'EB\',\n                                       \'EE\', \'EG\', \'EH\', \'EJ\', \'EL\', \'EP\', \'EU\', \'FC\', \'FD \', \'FE\', \'FI\', \'FL\',\n                                       \'FR\', \'FS\', \'GB\', \'GE\', \'GF\', \'GH\', \'GI\', \'GL\'],\n                         categorical_feature=

## 4) A pruner for LightGBM

In [40]:
'''
X = X_df.to_numpy()
y = y_df.to_numpy()

# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    param = {"objective": "binary",
             "metric": "binary_logloss",
             "verbosity": -1,
             "boosting_type": "gbdt",
             "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
             "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
             "num_leaves": trial.suggest_int("num_leaves", 2, 256),
             "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
             "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
             "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
             "seed": 42,
             "learning_rate":trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True),
            }

    for train_index, valid_index in skf.split(X, y):
        train_x, valid_x = X[train_index], X[valid_index]
        train_y, valid_y = y[train_index], y[valid_index]
    
        dtrain = lgb.Dataset(train_x,label=train_y,
                             feature_name=["AB", "AF", "AH", "AM", "AR", "AX", "AY", "AZ", "BC", "BD ", "BN", "BP",
                                           "BQ", "BR", "BZ", "CB", "CC", "CD ", "CF", "CH", "CL", "CR", "CS", "CU",
                                           "CW ", "DA", "DE", "DF", "DH", "DI", "DL", "DN", "DU", "DV", "DY", "EB",
                                           "EE", "EG", "EH", "EJ", "EL", "EP", "EU", "FC", "FD ", "FE", "FI", "FL",
                                           "FR", "FS", "GB", "GE", "GF", "GH", "GI", "GL"],
                             categorical_feature=["EJ"]
                            )
        dvalid = lgb.Dataset(valid_x,label=valid_y,
                             feature_name=["AB", "AF", "AH", "AM", "AR", "AX", "AY", "AZ", "BC", "BD ", "BN", "BP",
                                           "BQ", "BR", "BZ", "CB", "CC", "CD ", "CF", "CH", "CL", "CR", "CS", "CU",
                                           "CW ", "DA", "DE", "DF", "DH", "DI", "DL", "DN", "DU", "DV", "DY", "EB",
                                           "EE", "EG", "EH", "EJ", "EL", "EP", "EU", "FC", "FD ", "FE", "FI", "FL",
                                           "FR", "FS", "GB", "GE", "GF", "GH", "GI", "GL"],
                             categorical_feature=["EJ"]
                            )

        # Add a callback for pruning.
        # https://lightgbm.readthedocs.io/en/latest/Parameters.html#metric
        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "") 

        gbm = lgb.train(param, dtrain, valid_sets=[dvalid],feval=balanced_log_loss_eval,categorical_feature=["EJ"])

        preds = gbm.predict(valid_x)
        
        bll = balanced_log_loss(valid_y,preds)

        return bll

if __name__ == "__main__":
    # https://optuna.readthedocs.io/en/stable/faq.html#how-can-i-obtain-reproducible-optimization-results
    # Make the sampler behave in a deterministic way
    sampler = TPESampler(seed=42)  
    study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize",sampler=sampler)
    study.optimize(objective, n_trials=300)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
'''

'\nX = X_df.to_numpy()\ny = y_df.to_numpy()\n\n# FYI: Objective functions can take additional arguments\n# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).\ndef objective(trial):\n    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)\n    scores = []\n    \n    param = {"objective": "binary",\n             "metric": "binary_logloss",\n             "verbosity": -1,\n             "boosting_type": "gbdt",\n             "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),\n             "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),\n             "num_leaves": trial.suggest_int("num_leaves", 2, 256),\n             "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),\n             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),\n             "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),\n             "min_child_samples": trial.suggest_int("min_child_sampl

In [41]:
# Define the column transformer to perform encoding and imputation
preprocessor = ColumnTransformer(transformers=[('cat', 
                                                Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                                                ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]),
                                                [39]),  # Use integer column index instead of string column name
                                               ('num', 
                                                SimpleImputer(strategy='median'), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                                                   11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                                                                                   21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
                                                                                   31, 32, 33, 34, 35, 36, 37, 38, 40,
                                                                                   41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
                                                                                   51, 52, 53, 54, 55])  
                                              ],
                                 remainder='passthrough', sparse_threshold=0)

X = X_df.to_numpy()
y = y_df.to_numpy()

In [42]:
from sklearn.metrics import log_loss
from sklearn.utils.class_weight import compute_sample_weight

def objective(trial):
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []

    # https://lightgbm.readthedocs.io/en/latest/Parameters.html#objective-parameters
    param = {"objective": "binary",
             "metric": "binary_logloss",
             "verbosity": -1,
             "boosting_type": "gbdt",
             "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
             "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
             "num_leaves": trial.suggest_int("num_leaves", 2, 256),
             "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
             "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
             "pos_bagging_fraction": trial.suggest_float("pos_bagging_fraction", 0.4, 1.0),
             "neg_bagging_fraction": trial.suggest_float("neg_bagging_fraction", 0.4, 1.0),
             "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
             "seed": 42,
             "learning_rate":trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True),
             "is_unbalance":True,
            }

    for train_index, valid_index in skf.split(X, y):
        train_x, valid_x = X[train_index], X[valid_index]
        train_y, valid_y = y[train_index], y[valid_index]
        
        # Fit the preprocessor on the training data
        preprocessor.fit(train_x)
        
        # Transform the training and validation data
        train_x = preprocessor.transform(train_x)
        valid_x = preprocessor.transform(valid_x)
    
        dtrain = lgb.Dataset(train_x,label=train_y)
        dvalid = lgb.Dataset(valid_x,label=valid_y)

        # Add a callback for pruning.
        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "binary_logloss") 

        gbm = lgb.train(param, 
                        dtrain, valid_sets=[dvalid],
                        callbacks=[pruning_callback])

        preds = gbm.predict(valid_x)
        
        # https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_sample_weight.html#sklearn.utils.class_weight.compute_sample_weight
        # Calculate sample weights for an imbalanced dataset
        sample_weight = compute_sample_weight('balanced', valid_y)

        # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html#sklearn.metrics.log_loss
        # Calculate log loss with sample weights
        log_loss_weighted = log_loss(valid_y, preds,  
                                     eps=1e-15,
                                     sample_weight=sample_weight)
        
        return log_loss_weighted

if __name__ == "__main__":
    # https://optuna.readthedocs.io/en/stable/faq.html#how-can-i-obtain-reproducible-optimization-results
    # Make the sampler behave in a deterministic way
    sampler = TPESampler(seed=42)  
    study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize",sampler=sampler)
    study.optimize(objective, n_trials=300)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-07-30 19:55:02,681] A new study created in memory with name: no-name-7def33c1-6e8b-4446-9120-8239fd2ca20c
[I 2023-07-30 19:55:03,090] Trial 0 finished with value: 0.6398841686466716 and parameters: {'lambda_l1': 2.348881295853308e-05, 'lambda_l2': 3.6010467344475403, 'num_leaves': 188, 'feature_fraction': 0.759195090518222, 'bagging_fraction': 0.4936111842654619, 'bagging_freq': 2, 'pos_bagging_fraction': 0.4348501673009197, 'neg_bagging_fraction': 0.9197056874649611, 'min_child_samples': 62, 'learning_rate': 0.006796578090758156}. Best is trial 0 with value: 0.6398841686466716.
[I 2023-07-30 19:55:03,410] Trial 1 finished with value: 0.9544326794478387 and parameters: {'lambda_l1': 1.5320059381854043e-08, 'lambda_l2': 5.360294728728285, 'num_leaves': 214, 'feature_fraction': 0.5274034664069657, 'bagging_fraction': 0.5090949803242604, 'bagging_freq': 2, 'pos_bagging_fraction': 0.5825453457757226, 'neg_bagging_fraction': 0.7148538589793427, 'min_child_samples': 46, 'learning_rat

Number of finished trials: 300
Best trial:
  Value: 0.1667569163206571
  Params: 
    lambda_l1: 2.315160150066382e-06
    lambda_l2: 5.795473037427011e-07
    num_leaves: 129
    feature_fraction: 0.5381606680136374
    bagging_fraction: 0.7706871849373977
    bagging_freq: 2
    pos_bagging_fraction: 0.5569219616410732
    neg_bagging_fraction: 0.5874589935019283
    min_child_samples: 32
    learning_rate: 0.099787461889484


### Submission

In [43]:
'''
Number of finished trials: 200
Best trial:
  Value: 0.19853937238340558
  Params: 
    lambda_l1: 0.0006615357839140508
    lambda_l2: 5.444760538433432e-05
    num_leaves: 164
    feature_fraction: 0.6734791494978176
    bagging_fraction: 0.6488310314537731
    bagging_freq: 1
    pos_bagging_fraction: 0.653881084845293
    neg_bagging_fraction: 0.453129351074091
    min_child_samples: 31
    learning_rate: 0.09670147199850047

Number of finished trials: 300
Best trial:
  Value: 0.1667569163206571
  Params: 
    lambda_l1: 2.315160150066382e-06
    lambda_l2: 5.795473037427011e-07
    num_leaves: 129
    feature_fraction: 0.5381606680136374
    bagging_fraction: 0.7706871849373977
    bagging_freq: 2
    pos_bagging_fraction: 0.5569219616410732
    neg_bagging_fraction: 0.5874589935019283
    min_child_samples: 32
    learning_rate: 0.099787461889484
    
Number of finished trials: 400
Best trial:
  Value: 0.1667569163206571
  Params: 
    lambda_l1: 2.315160150066382e-06
    lambda_l2: 5.795473037427011e-07
    num_leaves: 129
    feature_fraction: 0.5381606680136374
    bagging_fraction: 0.7706871849373977
    bagging_freq: 2
    pos_bagging_fraction: 0.5569219616410732
    neg_bagging_fraction: 0.5874589935019283
    min_child_samples: 32
    learning_rate: 0.099787461889484
'''

'\nNumber of finished trials: 200\nBest trial:\n  Value: 0.19853937238340558\n  Params: \n    lambda_l1: 0.0006615357839140508\n    lambda_l2: 5.444760538433432e-05\n    num_leaves: 164\n    feature_fraction: 0.6734791494978176\n    bagging_fraction: 0.6488310314537731\n    bagging_freq: 1\n    pos_bagging_fraction: 0.653881084845293\n    neg_bagging_fraction: 0.453129351074091\n    min_child_samples: 31\n    learning_rate: 0.09670147199850047\n\nNumber of finished trials: 300\nBest trial:\n  Value: 0.1667569163206571\n  Params: \n    lambda_l1: 2.315160150066382e-06\n    lambda_l2: 5.795473037427011e-07\n    num_leaves: 129\n    feature_fraction: 0.5381606680136374\n    bagging_fraction: 0.7706871849373977\n    bagging_freq: 2\n    pos_bagging_fraction: 0.5569219616410732\n    neg_bagging_fraction: 0.5874589935019283\n    min_child_samples: 32\n    learning_rate: 0.099787461889484\n    \nNumber of finished trials: 400\nBest trial:\n  Value: 0.1667569163206571\n  Params: \n    lambda_l

In [44]:
# Get the best hyperparameters from the optimization process
best_params = study.best_params

# Fit the preprocessor on the entire training data
preprocessor.fit(X_df)

# Transform the training data
X_transformed = preprocessor.transform(X_df)

# Create a LightGBM dataset from the transformed training data
dtrain = lgb.Dataset(X_transformed, label=y_df)

# Merge the best_params dictionary with the force_col_wise parameter
params = {**best_params, **{'force_col_wise': True}}

# Train a LightGBM model using the best hyperparameters
gbm = lgb.train(params, dtrain)

#submission_df=test_df.drop(["Id"], axis=1)

# Preprocess the test data using the preprocessor fit on the training data
test_df_transformed = preprocessor.transform(test_df)

# Make predictions on the unseen data using the trained LightGBM model
preds = gbm.predict(test_df_transformed)

# The predictions are probabilities for class 1
prob_class_1 = preds

# To get probabilities for class 0, subtract from 1
prob_class_0 = 1 - prob_class_1

# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/412946
# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/409801
submission = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
submission['class_1'] = np.clip(prob_class_1,1e-15,1 - 1e-15)
submission['class_0'] = prob_class_0
submission.to_csv('submission.csv', index = False)

# Display the contents of the submission.csv
display(pd.read_csv('submission.csv'))

ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                  transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 [39]),
                                ('num', SimpleImputer(strategy='median'),
                                 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                  14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
                                  25, 26, 27, 28, 29, ...])])

[LightGBM] [Info] Total Bins 9469
[LightGBM] [Info] Number of data points in the train set: 617, number of used features: 57
[LightGBM] [Info] Start training from score 0.175041
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

,Id,class_0,class_1
0,00eed32682bb,0.525404,0.474596
1,010ebe33f668,0.525404,0.474596
2,02fa521e1838,0.525404,0.474596
3,040e15f562a2,0.525404,0.474596
4,046e85c7cc7f,0.525404,0.474596


In [45]:
'''
Id	class_0	class_1
0	00eed32682bb	0.525404	0.474596
1	010ebe33f668	0.525404	0.474596
2	02fa521e1838	0.525404	0.474596
3	040e15f562a2	0.525404	0.474596
4	046e85c7cc7f	0.525404	0.474596
'''

'\nId\tclass_0\tclass_1\n0\t00eed32682bb\t0.525404\t0.474596\n1\t010ebe33f668\t0.525404\t0.474596\n2\t02fa521e1838\t0.525404\t0.474596\n3\t040e15f562a2\t0.525404\t0.474596\n4\t046e85c7cc7f\t0.525404\t0.474596\n'

In [46]:
'''
# Get the best hyperparameters from the Optuna study
best_params = study.best_params

# Create an instance of the label encoder
le = LabelEncoder()

# Fit the label encoder on the training data
le.fit(X_df['EJ'])

# Transform the categorical feature in both the training and validation data
X_df['EJ'] = le.transform(X_df['EJ'])

# Create a LightGBM dataset from the entire training data
dtrain = lgb.Dataset(X_df, label=y_df,params={'force_col_wise': True})

# Merge the best_params dictionary with the force_col_wise parameter
params = {**best_params, **{'force_col_wise': True}}

# Train a LightGBM model using the merged parameters
gbm = lgb.train(params, dtrain)

submission_df=test_df.drop(["Id"], axis=1)
submission_df['EJ'] = le.transform(submission_df['EJ']) # Replaces the original categorical values with their corresponding integer encodings.

# Make predictions on unseen data
preds = gbm.predict(submission_df)

# The predictions are probabilities for class 1
prob_class_1 = preds

# To get probabilities for class 0, subtract from 1
prob_class_0 = 1 - prob_class_1

# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/412946
# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/409801
sample = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
sample['class_1'] = np.clip(prob_class_1,1e-15,1 - 1e-15)
sample['class_0'] = prob_class_0
sample.to_csv('submission.csv', index = False)

# Display the contents of the submission.csv
display(pd.read_csv('submission.csv'))
'''

'\n# Get the best hyperparameters from the Optuna study\nbest_params = study.best_params\n\n# Create an instance of the label encoder\nle = LabelEncoder()\n\n# Fit the label encoder on the training data\nle.fit(X_df[\'EJ\'])\n\n# Transform the categorical feature in both the training and validation data\nX_df[\'EJ\'] = le.transform(X_df[\'EJ\'])\n\n# Create a LightGBM dataset from the entire training data\ndtrain = lgb.Dataset(X_df, label=y_df,params={\'force_col_wise\': True})\n\n# Merge the best_params dictionary with the force_col_wise parameter\nparams = {**best_params, **{\'force_col_wise\': True}}\n\n# Train a LightGBM model using the merged parameters\ngbm = lgb.train(params, dtrain)\n\nsubmission_df=test_df.drop(["Id"], axis=1)\nsubmission_df[\'EJ\'] = le.transform(submission_df[\'EJ\']) # Replaces the original categorical values with their corresponding integer encodings.\n\n# Make predictions on unseen data\npreds = gbm.predict(submission_df)\n\n# The predictions are probabil

# References/Resources
- https://www.kaggle.com/competitions/icr-identify-age-related-conditions/overview
- https://optuna.readthedocs.io/en/stable/index.html
- https://lightgbm.readthedocs.io/en/latest/index.html
- https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
- https://github.com/optuna/optuna-examples/blob/main/lightgbm/lightgbm_simple.py
- https://github.com/optuna/optuna-examples/blob/main/lightgbm/lightgbm_tuner_simple.py
- https://github.com/optuna/optuna-examples/blob/main/lightgbm/lightgbm_tuner_cv.py
- https://github.com/optuna/optuna-examples/blob/main/lightgbm/lightgbm_integration.py
- https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/422442
- https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/422194
- https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/412946
- https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/409801
- https://chat.openai.com/
- https://bard.google.com/
- https://www.bing.com/search?q=Bing+AI&showconv=1&FORM=hpcodx&sydconv=1